In [1]:
from huggingface_hub import login
import os

In [ ]:

hf_token = os.getenv("HF_TOKEN")

if hf_token:
    login(token=hf_token)
    print("Logged in successfully using HF_TOKEN environment variable.")

Logged in successfully using HF_TOKEN environment variable.


In [ ]:
from huggingface_hub import HfApi

api = HfApi(token=hf_token)
api.upload_folder(
    folder_path="./gpt2-it-ticket-classifier",
    repo_id="parth1609/gpt2-it-ticket-classifier",
    repo_type="model",
)


KeyboardInterrupt: 

In [ ]:
import requests
import os

# --- Configuration ---
# Your model ID on Hugging Face Hub
MODEL_ID = "parth1609/gpt2-it-ticket-classifier"

# Your Hugging Face API token
# It's highly recommended to load this from an environment variable for security
# Make sure you've set HF_TOKEN="hf_YOUR_TOKEN_HERE" in your environment
# API_TOKEN = os.getenv("HF_TOKEN")
API_TOKEN = ""

if not API_TOKEN:
    raise ValueError("Hugging Face API token not found. Please set the HF_TOKEN environment variable.")

# The Inference API URL for your model
API_URL = f"https://api-inference.huggingface.co/models/{MODEL_ID}"

# Headers for authentication
headers = {
    "Authorization": f"Bearer {API_TOKEN}",
    "Content-Type": "application/json" # Specify content type for JSON payload
}

# --- Function to send a query to the API ---
def query(payload):
    """
    Sends a query (input text) to the Hugging Face Inference API.

    Args:
        payload (dict): A dictionary containing the input data for the model.
                        For text models, it's typically {"inputs": "Your input text"}.

    Returns:
        dict: The JSON response from the API, containing the model's predictions.
    """
    response = requests.post(API_URL, headers=headers, json=payload)
    response.raise_for_status() # Raises an HTTPError for bad responses (4xx or 5xx)
    return response.json()


In [5]:

sample = [
    "My laptop is not turning on after the latest update.",
    "I need help resetting my VPN password, it keeps failing.",
    "The software update caused my computer to crash.",
]


In [4]:
from transformers import pipeline
import torch

# initialize pipeline with your model
pipe = pipeline(
    "text2text-generation",
    model=MODEL_ID,
    tokenizer=MODEL_ID,
    device=0 if torch.cuda.is_available() else -1  # use GPU if available
)

# example ticket list
sample_tickets = [
    "My laptop is not turning on after the latest update.",
    "The VPN disconnects every time I upload files.",
    "Wi-Fi drops every morning at 9 AM in the office."
]

for ticket in sample_tickets:
    prompt = (
        "Classify the following IT ticket and return output in JSON format with keys:\n"
        "'ticket', 'Department', 'Priority'.\n\n"
        f"Ticket: \"{ticket}\""
    )

    # include generation settings
    output = pipe(prompt, max_new_tokens=128, do_sample=False)
    text = output[0]["generated_text"].strip()

    print("🔹 Ticket:")
    print(ticket)
    print("✅ Model Output:")
    print(text)
    print("-" * 60)


Device set to use cuda:0
The model 'GPT2LMHeadModel' is not supported for text2text-generation. Supported models are ['PeftModelForSeq2SeqLM', 'BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'GraniteSpeechForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UM

🔹 Ticket:
My laptop is not turning on after the latest update.
✅ Model Output:
Classify the following IT ticket and return output in JSON format with keys:
'ticket', 'Department', 'Priority'.

Ticket: "My laptop is not turning on after the latest update."   {"department": "Returns and Exchanges", "priority": "medium"}   {"department": "Returns and Exchanges", "priority": "medium"}   {"department": "Returns and Exchanges", "priority": "medium"}   {"department": "Returns and Exchanges", "priority": "medium"}   {"department": "Returns and Exchanges", "priority": "medium"}   {"department": "Returns and Exchanges", "priority": "medium"}   {"department":
------------------------------------------------------------
🔹 Ticket:
The VPN disconnects every time I upload files.
✅ Model Output:
Classify the following IT ticket and return output in JSON format with keys:
'ticket', 'Department', 'Priority'.

Ticket: "The VPN disconnects every time I upload files."   {"department": "Returns and Exchange